In [79]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 50

In [80]:
restaurants_df = pd.read_excel(r'restaurants_clean_encoded2.xlsx')

In [87]:
restaurants_df.corr()

,Price Range,target_value_fix,Number of Reviews,Amsterdam,Athens,Barcelona,Berlin,Bratislava,Brussels,Budapest,...,Sushi,Soups,Pizza,Fusion,Gluten Free Options,Halal,Healthy,Kosher,Vegan Options,Vegetarian Friendly
Price Range,1.000000,0.155682,0.162342,0.007798,-0.031050,-0.019362,-0.033952,0.004855,0.019562,-0.024532,...,0.047483,-0.030388,-0.061601,0.049224,0.204001,-0.034505,-0.041613,0.002602,0.048190,NaN
target_value_fix,0.155682,1.000000,0.302613,-0.007270,0.015360,0.002398,0.040905,0.020477,-0.019422,-0.001065,...,-0.019950,0.049387,-0.045031,0.047789,0.368368,0.009319,0.025621,-0.008237,0.350119,NaN
Number of Reviews,0.162342,0.302613,1.000000,-0.004061,-0.012538,0.013575,-0.039025,-0.017619,-0.022547,0.010561,...,-0.010482,0.090822,0.033275,0.007893,0.362280,-0.011237,-0.022042,0.010075,0.266750,NaN
Amsterdam,0.007798,-0.007270,-0.004061,1.000000,-0.023222,-0.049819,-0.040510,-0.013966,-0.030698,-0.027630,...,-0.005948,-0.003171,-0.012570,-0.002192,0.024286,-0.003948,0.014660,-0.001523,0.022503,NaN
Athens,-0.031050,0.015360,-0.012538,-0.023222,1.000000,-0.034239,-0.027841,-0.009598,-0.021098,-0.018989,...,-0.013607,-0.011678,-0.024121,-0.013506,-0.016645,-0.014098,-0.001413,-0.003272,0.026438,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Halal,-0.034505,0.009319,-0.011237,-0.003948,-0.014098,-0.016968,-0.002222,-0.011190,0.000712,-0.013874,...,-0.029598,-0.018868,-0.039522,-0.014111,0.016102,1.000000,-0.007416,-0.002466,0.076935,NaN
Healthy,-0.041613,0.025621,-0.022042,0.014660,-0.001413,0.033225,-0.005123,0.011703,0.003404,0.008716,...,-0.010802,0.062370,-0.029645,0.032368,0.052619,-0.007416,1.000000,-0.004383,0.089189,NaN
Kosher,0.002602,-0.008237,0.010075,-0.001523,-0.003272,-0.008631,0.002929,-0.003419,-0.007515,0.011604,...,0.000276,0.010885,-0.003083,-0.004605,0.001297,-0.002466,-0.004383,1.000000,0.021456,NaN
Vegan Options,0.048190,0.350119,0.266750,0.022503,0.026438,0.020936,0.006994,-0.012115,-0.023545,0.005404,...,0.003803,0.067117,0.056662,0.045597,0.468830,0.076935,0.089189,0.021456,1.000000,NaN


In [88]:
RAND_STATE = 34 # for reproducible shuffling
TT_RATIO = 0.3 # test/train

In [89]:
X = restaurants2_df.drop('target_value_fix', axis=1)
y = restaurants2_df.target_value_fix

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TT_RATIO, random_state=RAND_STATE)
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
X_train.head()

,Price Range,Amsterdam,Athens,Barcelona,Berlin,Bratislava,Brussels,Budapest,Copenhagen,Dublin,...,Sushi,Soups,Pizza,Fusion,Gluten Free Options,Halal,Healthy,Kosher,Vegan Options,Vegetarian Friendly
31364,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
34775,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
48526,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
6900,2,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
37164,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1


In [91]:
print(X_train.shape)
print(X_test.shape)

(51957, 159)
(22268, 159)


In [92]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [93]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

(51957, 159)
(22268, 159)


In [94]:
model = sm.OLS(y_train, X_train_scaled).fit()
predictions_train = model.predict(X_train_scaled)

predictions_test = model.predict(X_test_scaled)
print_model = model.summary()
print(print_model)

                                 OLS Regression Results                                
Dep. Variable:       target_value_fix   R-squared (uncentered):                   0.057
Model:                            OLS   Adj. R-squared (uncentered):              0.054
Method:                 Least Squares   F-statistic:                              20.09
Date:                Sun, 16 Apr 2023   Prob (F-statistic):                        0.00
Time:                        20:40:46   Log-Likelihood:                     -1.2738e+05
No. Observations:               51957   AIC:                                  2.551e+05
Df Residuals:                   51802   BIC:                                  2.564e+05
Df Model:                         155                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [95]:
X_train_const_scaled = sm.add_constant(X_train_scaled) # adding a constant
X_test_const_scaled = sm.add_constant(X_test_scaled) # adding a constant

In [98]:
print(X_train_const_scaled.shape)
print(X_test_const_scaled.shape)

(51957, 160)
(22268, 159)


In [99]:
print(X_train_const_scaled.shape)
print(X_test_const_scaled.shape)

(51957, 160)
(22268, 159)


In [100]:
model=LinearRegression() 
model.fit(X_train_scaled, y_train)

LinearRegression()

In [101]:
model.coef_

array([ 2.44771654e-02,  7.76290366e+10,  5.49164061e+10,  1.10530910e+11,
        9.21200919e+10,  3.26299275e+10,  7.14584892e+10,  6.33601127e+10,
        5.68264255e+10,  6.01010275e+10,  5.93304901e+10,  4.95833124e+10,
        5.56888250e+10,  4.28957251e+10,  4.60546156e+10,  7.76733306e+10,
        2.90261716e+10,  1.59340301e+11,  3.31826318e+10,  5.92408851e+10,
        1.06377837e+11,  9.60728979e+10,  6.41878874e+10,  5.05490699e+10,
        4.40568788e+10,  1.44307916e+11,  7.88367467e+10,  1.05649365e+11,
        5.86095094e+10,  7.13123363e+10,  5.60073074e+10,  5.17726889e+10,
        2.90737152e-02,  2.65121460e-03,  8.51716995e-02,  5.27191162e-03,
       -5.37986755e-02,  2.19879150e-02,  5.49945889e+11,  1.66511536e-02,
       -4.81090546e-02,  3.92780304e-02,  2.40659714e-03,  1.16329193e-02,
       -1.48086548e-02,  7.85064697e-03, -2.53009796e-03, -8.43811035e-03,
       -1.59692764e-03,  1.35641098e-02,  1.30767822e-02, -1.81541443e-02,
        5.23567200e-03,  

In [102]:
model.intercept_

2.6319579411447744

In [103]:
y_pred = pd.DataFrame(model.predict(X_test_scaled),columns = ['target_value_fix'] )
y_pred_train =  pd.DataFrame(model.predict(X_train_scaled),columns = ['target_value_fix'])

In [104]:
R2=r2_score(y_test,y_pred)
R2

0.3247760478886197